In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Times New Roman'
import seaborn as sns
sns.set_color_codes()
import pandas as pd
import numpy as np
import os
import math
from matplotlib.ticker import FuncFormatter

In [2]:
# # ATSC Grid
prefix = 'atsc_kao_net'
# # ATSC Monaco
# prefix = 'atsc_real_net'
table_names = ['traffic', 'trip', 'control']
test_dir = 'training/'
test_map = 'kao/'
parameters = 'extend_range/'
algo = 'ma2c_nclm'
def load_summarize_data():
    data = {}
    print(algo)
    data[algo] = {}
    for tab in table_names:
        data_dir = test_dir + test_map + parameters + algo + '/eva_data/' + ('%s_%s_%s.csv' % (prefix, "ma2c_nclm", tab))
        df = pd.read_csv(data_dir)
        if tab == 'control':
            col_names = ['reward']
        elif tab == 'traffic':
            col_names = ['avg_queue', 'avg_speed_mps', 'avg_wait_sec', 'number_arrived_car']
        else:
            col_names = ['wait_sec']
        print_metrics(df, col_names, tab)
        data[algo][tab] = df
    return data  
            
def print_metrics(df, col_names, tab_name):
    if tab_name == 'control':
        rewards = df.groupby(['episode']).reward.mean()
        mean_reward = np.mean(rewards.values)
        std_reward = np.std(rewards.values)
        print('R_bar: mean %.2f, std %.2f' % (mean_reward, std_reward))
    episodes = list(df.episode.unique())
    num_episode = len(episodes)
    for name in col_names:
        if tab_name != 'trip':
            res = df.loc[df.episode == episodes[0], name].values
            for episode in episodes[1:]:
                res += df.loc[df.episode == episode, name].values
            res = res / num_episode
            print('%s: mean %.2f, min %.2f, max: %.2f' % (name, np.mean(res), np.min(res), np.max(res)))
        else:
            res = []
            for episode in episodes:
                res += list(df.loc[df.episode == episode, name].values)
            print('%s: mean %d, max %d' % (name, np.mean(res), np.max(res)))
            
data = load_summarize_data()

ma2c_nclm
avg_queue: mean 0.63, min 0.00, max: 0.97
avg_speed_mps: mean 1.63, min 0.00, max: 12.36
avg_wait_sec: mean 125.90, min 0.00, max: 192.67
number_arrived_car: mean 0.24, min 0.00, max: 0.64
wait_sec: mean 478, max 3330
R_bar: mean -277.03, std 106.47
reward: mean -277.03, min -708.18, max: 210.18
